[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain 学院课程跟学

## 聊天模型

在这个课程中，我们将使用 [聊天模型](https://python.langchain.com/v0.2/docs/concepts/#chat-models)，这些模型做一些事情，即接收一系列消息作为输入，并返回聊天消息作为输出。 LangChain 不托管任何聊天模型，而是依赖于第三方集成。[这里](https://python.langchain.com/v0.2/docs/integrations/chat/) 是 LangChain 中的第三方聊天模型集成列表！默认情况下，课程将使用 [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) ，因为它既受欢迎又性能良好。如上所述，请确保您有一个 OPENAI_API_KEY 。

这里因为我没有openai的key，我用qwen兼容openai的api的模型，所以通过配置文件指定api_key和base_url

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [10]:
import yaml

with open('../.yml', 'r') as file:
    config = yaml.safe_load(file)

# 获取配置信息(以.yml_example的例子)
qwen_config = config['llm']['qwen']

[这里](https://python.langchain.com/v0.2/docs/how_to/#chat-models) 有一个关于如何使用聊天模型的实用指南，但我们将在下面展示一些亮点。如果你已经运行了 `pip install -r requirements.txt` 那么你已经安装了 `langchain-openai`包。有了这个，我们可以实例化我们的 `ChatOpenAI`模型对象。

我们可以设置[一些标准参数](https://python.langchain.com/v0.2/docs/concepts/#chat-models) 来调整聊天模型。其中2个最常见的是：

* `model`: 模型的名称
* `temperature`: 样本温度

`Temperature` 控制模型输出的随机性或创造性，低温度（接近 0）更确定性和专注，适用于需要准确或事实性回答的任务。高温度（接近 1）适用于创造性任务或生成多样化的回答。

In [3]:
from langchain_openai import ChatOpenAI

# 注意我没有设置环境OPENAI_API_KEY，所以通过参数指定了api_key和base_url，如果你有OPENAI_API_KEY，可以不指定
qwlong_chat = ChatOpenAI(model="qwen-long", temperature=0, api_key=qwen_config['api_key'], base_url=qwen_config['base_url'])
qwmax_chat = ChatOpenAI(model="qwen-max", temperature=0, api_key=qwen_config['api_key'], base_url=qwen_config['base_url'])

LangChain 中的聊天模型有[多个默认方法](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface)。大部分情况下，我们将使用：

* `stream`: 将响应的块流回
* `invoke`: 在输入上调用链

如前所述，聊天模型将[消息](https://python.langchain.com/v0.2/docs/concepts/#messages) 作为输入。消息有一个角色（描述说消息的人）和一个内容属性。我们将在稍后更多地讨论这一点，但在这里，让我们先展示基本用法。

In [4]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
qwlong_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today? If you have any questions or need help with coding, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 2, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-long', 'system_fingerprint': None, 'id': 'chatcmpl-2a0cf72e-9142-9bc9-b63e-d87c0fc00e97', 'finish_reason': 'stop', 'logprobs': None}, id='run-67984051-6e0b-4069-a918-9ee401bb2d1a-0', usage_metadata={'input_tokens': 2, 'output_tokens': 25, 'total_tokens': 27, 'input_token_details': {}, 'output_token_details': {}})

我们得到一个`AIMessage`响应。另外，注意我们只需用一个字符串来调用聊天模型。当传入字符串作为输入时，它会被转换为`HumanMessage`，然后传递给底层模型。


In [6]:
qwlong_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today? If you have any questions or need help with coding, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 2, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-long', 'system_fingerprint': None, 'id': 'chatcmpl-0b0d4237-3db6-9b45-820e-c5e3c26a6d01', 'finish_reason': 'stop', 'logprobs': None}, id='run-84caf022-be83-4787-9d65-521eff584d0a-0', usage_metadata={'input_tokens': 2, 'output_tokens': 25, 'total_tokens': 27, 'input_token_details': {}, 'output_token_details': {}})

In [7]:
qwmax_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 10, 'total_tokens': 19, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'id': 'chatcmpl-04b7364b-7164-9ef2-9705-71254ce586c6', 'finish_reason': 'stop', 'logprobs': None}, id='run-8f310a61-a46d-4b49-9eac-909400bd4025-0', usage_metadata={'input_tokens': 10, 'output_tokens': 9, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

界面在所有聊天模型中保持一致，模型通常在每个笔记本启动时初始化一次。

因此，如果您对另一个提供商有强烈的偏好，可以轻松地在模型之间切换，而无需更改下游代码。


## 搜索工具

[Tavily](https://tavily.com/) , 这是一个针对LLMs和 RAG 优化的搜索引擎，旨在实现高效、快速和持久的搜索结果。如前所述，注册简单，并提供慷慨的免费层。一些课程（在第 4 模块中）将默认使用 Tavily，但当然，如果您想自己修改代码，也可以使用其他搜索工具。

In [11]:
tavily_config = config['tavily']

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults


tavily_search = TavilySearchResults(max_results=3, tavily_api_key=tavily_config['key'])
search_docs = tavily_search.invoke("What is LangGraph?")

In [13]:
search_docs

[{'title': 'What is LangGraph? - Analytics Vidhya',
  'url': 'https://www.analyticsvidhya.com/blog/2024/07/langgraph-revolutionizing-ai-agent/',
  'content': 'To sum up, LangGraph is a major advancement in the development of AI agents. It enables developers to push the limits of what’s possible with AI agents by eliminating the shortcomings of earlier systems and offering a flexible, graph-based framework for agent construction and execution. LangGraph is positioned to influence the direction of artificial intelligence significantly in the future. [...] LangGraph is a library built on top of Langchain that is designed to facilitate the creation of cyclic graphs for large language model (LLM) – based AI agents.\nIt views agent Objective Points about LangGraph and workflows as cyclic graph topologies, allowing for more variable and nuanced agent behaviors than linear execution models. [...] Frameworks such as LangGraph are becoming increasingly important as AI develops. LangGraph is maki